# Research Assistant


## Install Required Packages
We use Crewai, Composio and Langchain to build a research assistant agent.

In [ ]:
%pip install composio_crewai
%pip install crewai
%pip install langchain-openai

You need to connect your SerpAPI account using the Composio CLI. This allows agents to use SerpAPI for web searches. Run the following commands on your terminal:

In [ ]:
%composio login
%composio add serpapi

## Importing required libraries


In [ ]:
import os
import dotenv
from crewai import Agent, Task, Crew, Process
from composio_langchain import ComposioToolSet, Action, App
from langchain_openai import ChatOpenAI

## Initialise LLM

In [ ]:
dotenv.load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("The OPENAI_API_KEY environment variable is not set.")
llm = ChatOpenAI(openai_api_key=openai_api_key, model_name="gpt-4o")

## Initialise Tools

In [ ]:
# Composio tool for serpapi
composio_toolset = ComposioToolSet()
tools = composio_toolset.get_tools(actions=[Action.SERPAPI_SEARCH])

## Initialise Agent

In [ ]:
researcher = Agent(
     role='Researcher',
     goal='Search the internet for the information requested',
     backstory="""
     You are a researcher. Using the information in the task, you find out some of the most popular facts about the topic along with some of the trending aspects.
     You provide a lot of information thereby allowing a choice in the content selected for the final blog.
     """,
     verbose=True,
     allow_delegation=False,
     tools=tools,
     llm=llm
 )


## Define the task and execute

In [ ]:
task1 = Task( description="""Research about open source LLMs vs
    closed source LLMs. Your final answer MUST be a full analysis report""", #You can add your own topic to research about change the topic, edit the text after 'Research about' in the description parameter of task1 
    expected_output='When the research report is ready',
    agent=researcher
)
task1.execute()


## Putting it all Together

In [ ]:
from crewai import Agent, Task, Crew, Process
from composio_langchain import ComposioToolSet, App
from langchain_openai import ChatOpenAI
import os
import dotenv

# Load environment variables
dotenv.load_dotenv()

# Initialize the language model with OpenAI API key and model name
llm = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model_name="gpt-4o"
)

# Setup tools using ComposioToolSet
composio_toolset = ComposioToolSet()
tools = composio_toolset.get_tools(apps=[App.SERPAPI])

# Define the Researcher agent with its role, goal, and backstory
researcher = Agent(
    role='Researcher',
    goal='Search the internet for the information requested',
    backstory="""
    You are a researcher. Using the information in the task, you find out some of the most popular facts about the topic along with some of the trending aspects.
    You provide a lot of information thereby allowing a choice in the content selected for the final blog.
    """,
    verbose=True,  # Enable verbose logging for the agent
    allow_delegation=False,  # Disable delegation
    tools=tools,  # Assign the tools to the agent
    llm=llm  # Assign the language model to the agent
)

# Define the research task with its description and expected output
task1 = Task(
    description="""
    Research about open source LLMs vs closed source LLMs.
    Your final answer MUST be a full analysis report
    """, #you can add your own topic after "Research about {your topic}"
    expected_output='When the research report is ready',  # Define the expected output
    agent=researcher  # Assign the task to the researcher agent
)

# Execute the task
task1.execute()

# Print the result of the task execution
print(task1.result)
